

## Scraping μιας σελίδας του Yelp

Πάμε να δοκιμάσουμε να εξορύξουμε περιεχόμενο από την σελίδα Yelp. 
Η σελίδα που θέλουμε να πάρουμε είναι η τοποθεσία του Burger House "Shake Shack" [this Shake Shack location](https://www.yelp.com/biz/shake-shack-new-york-54).

Θα πάρουμε το περιεχόμενο της html και στην συνέχεια θα αποθηκεύσουμε τις πληροφορίες σε ένα dictionary.

In [ ]:
#Καλούμε το selenium και ζητάμε να ανοίξει τον browser για μας
from selenium import webdriver

driver = webdriver.Chrome()

driver.get('https://www.yelp.com/biz/shake-shack-new-york-54')

store_name = driver.find_element_by_class_name("biz-page-title").text
full_address = driver.find_element_by_class_name("street-address").text
stars = driver.find_element_by_class_name("i-stars").get_attribute('title')
categories = driver.find_element_by_class_name("category-str-list").text

store = {
    'name': store_name,
    'address': full_address,
    'stars': stars,
    'categories': categories
}

store

## Scraping για πολλές σελίδες στο Yelp

Μερικές φορές δεν χρειαζόμαστε μόνο *μια* σελίδα, αλλά *πολλές* σελίδες. 

Σε αυτή την περίπτωση χρειαζόμαστε 2 πράγματα:

1. Ένα από το οποίο θα κάνετε scraping για κάθε γραμμή.
2. Ένα function για το scraping

### Το dataframe μας

In [ ]:
import pandas as pd

df = pd.read_csv("../../data/yelp_sample.csv")
df.head(3)

### To function  για το scraping

Θα πάρουμε τον κώδικα που έχουμε γράψει επάνω και θα κάνουμε κάποιες μικρές αλλαγές:

### 1. Θα προσθέσουμε το `def κάνεκάτισεκάθε(row)` για να το μετατρέψουμε σε ένα function

Επειδή είναι function, θα χρειαστεί να βάλουμε indent.

Επίσης, πρέπει να προσέξουμε πολύ να ΜΗΝ έχουμε το `driver = webdriver.Chrome()` μέσα στο function, γιατί θα ανοίγει ένα νέο παράθυρο στον Chrome κάθε φορά που θα επισκέπτεται μία σελίδα!!!

### 2. Χρησιμοποιήστε τη μεταβλητή `row`, ώστε να μην παίρνουμε την ίδια σελίδα κάθε φορά.


**Παλιός κώδικας**

```python
driver.get("https://www.yelp.com/biz/shake-shack-new-york-54")
```

Τώρα έχω τη μεταβλητή `row` που είναι μια γραμμή από τα δεδομένα μας. Αν θέλουμε να φτιάξουμε ένα URL, παίρνουμε το `"https://www.yelp.com/biz/"` και προσθέτουμε το `row['slug']` (που στην ουσία είναι ο τρόπος που το Yelp δημιουργεί τα URLs).

**Nέος κώδικας**

```python
driver.get("https://www.yelp.com/biz/" + row['slug'])
```

### 3. Επιστρέψτε ένα `pd.Series` των δεδομένων μας αντί για ένα λεξικό (dictionary).

**Παλιός κώδικας**

```python
store = {
    'name': store_name,
    'address': full_address,
    'stars': stars,
    'categories': categories
}

store
```

Επειδή είναι function, χρειαζόμαστε το **return** για να επιστρέψει κάτι - και για να προστεθούν οι νέες στήλες στο dataframe, πρέπει να είναι `pd.Series`.

**Nέος κώδικας**

```python
return pd.Series({
    'name': store_name,
    'address': full_address,
    'stars': stars,
    'categories': categories
})
```

## Ολοκληρωμένος ο παλιός κώδικας

In [ ]:
driver = webdriver.Chrome()

driver.get("https://www.yelp.com/biz/shake-shack-new-york-54")

store_name = driver.find_element_by_class_name("biz-page-title").text
full_address = driver.find_element_by_class_name("street-address").text
stars = driver.find_element_by_class_name("i-stars").get_attribute('title')
categories = driver.find_element_by_class_name("category-str-list").text

store = {
    'name': store_name,
    'address': full_address,
    'stars': stars,
    'categories': categories
}

store

## Ολοκληρωμένος ο νέος κώδικας!

In [ ]:
def get_yelp_info(row):
    driver.get("https://www.yelp.com/biz/" + row['slug'])

    store_name = driver.find_element_by_class_name("biz-page-title").text
    full_address = driver.find_element_by_class_name("street-address").text
    stars = driver.find_element_by_class_name("i-stars").get_attribute('title')
    categories = driver.find_element_by_class_name("category-str-list").text

    return pd.Series({
        'store_name': store_name,
        'address': full_address,
        'stars': stars,
        'categories': categories
    })

## Χρησιμοποιώντας το function

Τώρα που έχουμε το function έτοιμο, πρέπει να το **χρησιμοποιήσουμε**!

1. Ανοίξτε τον `driver`
2. Χρησιμοποιήστε το `.apply` για να τρέξει το function σε κάθε γραμμή (row)
3. Χρησιμοποιήστε το `.join` για να προσθέσετε τις στήλες στο dataframe

Ουσιαστικά πάντα θα αντιγράφετε αυτόν τον κώδικα. 

**Σιγουρευτείτε ότι αλλάζετε τα ονόματα της μεταβλητής.**

In [ ]:
# Άνοιξε έναν καινούριο Chrome
driver = webdriver.Chrome()

# Πάρε κάθε row και βάλτο μέσα στο get_yelp_info και συνδύασέ το με τα παλιά δεδομένα.
new_df = df.apply(get_yelp_info, axis=1).join(df)
new_df.head()

## Τι συμβαίνει με το BeautifulSoup?

Κανένα πρόβλημα, μπορούμε να κάνουμε  *ακριβώς το ίδιο πράγμα.* Δεν έχει να κάνει με το Selenium, έχει να κάνει με τα pandas!

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
def get_yelp_with_bs(row):
    response = requests.get("https://www.yelp.com/biz/" + row['slug'])
    doc = BeautifulSoup(response.text, 'html.parser')
    
    store_name = doc.find(class_="biz-page-title").text
    full_address = doc.find(class_="street-address").text
    stars = doc.find(class_="i-stars")['title']
    categories = doc.find(class_="category-str-list").text

    return pd.Series({
        'store_name': store_name,
        'address': full_address,
        'stars': stars,
        'categories': categories
    })

In [ ]:
bs_df = df.apply(get_yelp_with_bs, axis=1).join(df)
bs_df.head()

## BONUS: Πόσο πιο γρήγορο είναι το Selenium σε σχέση με το BeautifulSoup?

Μπορούμε να χρησιμοποιήσουμε το `%%time` και να τα χρονομετρήσουμε

**ΞΕΚΙΝΑΜΕ!**

Ας δούμε το **Selenium:**

In [ ]:
%%time
new_df = df.apply(get_yelp_info, axis=1).join(df)
new_df.head()

Ας δοκιμάσουμε το **BeautifulSoup και requests:**

In [ ]:
%%time
bs_df = df.apply(get_yelp_with_bs, axis=1).join(df)
bs_df.head(2)

Δεν έχουν μεγάλη διαφορά σε αυτή την περίπτωση!